

---


# 00. About week4


---



1.   Importing libraries
2.   Linear regression
2.   Multiclass classification
3.   Binary classification



---


# 01. Importing libraries


---



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
print(tf.__version__)



---


# 02. Linear regression


---



In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df_onehot = pd.get_dummies(df, columns=['sex', 'smoker'], drop_first=True, dtype = int)
df_onehot = pd.get_dummies(df_onehot, columns=['region'], drop_first=False, dtype = int)
df_onehot.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

min_max_scaler = MinMaxScaler()
robust_scaler = RobustScaler()


df_onehot['age'] = robust_scaler.fit_transform(df_onehot['age'].values.reshape(-1, 1))
df_onehot['bmi'] = min_max_scaler.fit_transform(df_onehot['bmi'].values.reshape(-1, 1))
df_onehot['children'] = min_max_scaler.fit_transform(df_onehot['children'].values.reshape(-1, 1))
df_onehot.head()

In [ ]:
import seaborn as sns

correlation_matrix = df_onehot.corr()


plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
x = df_onehot.drop('charges', axis=1).values
y = df_onehot['charges'].values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
x_train_tensor = tf.constant(x_train, dtype=tf.float32)
y_train_tensor = tf.constant(y_train, dtype=tf.float32)
x_test_tensor = tf.constant(x_test, dtype=tf.float32)
y_test_tensor = tf.constant(y_test, dtype=tf.float32)

In [ ]:
x_train_tensor.shape, y_train_tensor.shape, x_test_tensor.shape, y_test_tensor.shape

In [ ]:
model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(9,)),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
model_1.summary()

In [ ]:
model_1.compile(loss=tf.keras.losses.mae,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                metrics=['mape'])

In [ ]:
history_1 = model_1.fit(x_train_tensor, y_train_tensor, epochs=100, verbose=2)

In [ ]:
pd.DataFrame(history_1.history).plot()

In [ ]:
model_1.evaluate(x_test_tensor, y_test_tensor)

In [ ]:
y.min(), y.max()

In [ ]:
tf.keras.metrics.MeanAbsolutePercentageError()(y_test_tensor, model_1.predict(x_test_tensor))

In [ ]:
tf.keras.metrics.MeanAbsoluteError()(y_test_tensor, model_1.predict(x_test_tensor))



---


# 03. Multiclass classification


---



In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
type(x_train), type(y_train), type(x_test), type(y_test)

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
np.unique(y_train)

In [ ]:
img=x_test[30]
plt.imshow(img, cmap='gray')

In [ ]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
x_train_tensor = tf.constant(x_train, dtype=tf.float32)
y_train_tensor = tf.constant(y_train, dtype=tf.float32)
x_test_tensor = tf.constant(x_test, dtype=tf.float32)
y_test_tensor = tf.constant(y_test, dtype=tf.float32)

In [ ]:
model_2 = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model_2.summary()

In [ ]:
model_2.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
history_2 = model_2.fit(x_train_tensor, y_train_tensor, epochs=10, verbose=1)

In [ ]:
pd.DataFrame(history_2.history).plot()

In [ ]:
model_2.evaluate(x_test_tensor, y_test_tensor)

In [ ]:
pred = model_2.predict(x_test_tensor[:14])
fig = plt.figure(figsize=(10, 4))
for i in range(14):
  fig.add_subplot(2, 7, i+1)
  plt.imshow(x_test[i], cmap='gray')
  plt.title(f"Predict:  {np.argmax(pred[i])}")



---


# 04. Binary classification


---



In [ ]:
data, ds_info = tfds.load(
    "cats_vs_dogs",
    split=["train"],
    shuffle_files = True,
    as_supervised = True,
    with_info = True
)

In [ ]:
ds_info.features

In [ ]:
class_names = ds_info.features["label"].names
class_names

In [ ]:
DATASET_SIZE = 23262

train_size = int(0.8 * DATASET_SIZE)

full_data = data[0].shuffle(buffer_size=100)
train_data = full_data.take(train_size)
test_data = full_data.skip(train_size)

In [ ]:
sample_img = test_data.take(1)
for image, label in sample_img:
    print(f"""
    Image shape: {image.shape}
    Image dtype: {image.dtype}
    Target class: {label}
    Class name: {class_names[label.numpy()]}
    """)

In [ ]:
tf.reduce_min(image), tf.reduce_max(image)

In [ ]:
plt.imshow(image)

In [ ]:
def preprocess_img(image, label, img_shape=300, rescale=False):
    image = tf.image.resize(image, [img_shape, img_shape])
    if rescale==False:
        return tf.cast(image, tf.float32), label
    else:
        return tf.cast(image, tf.float32)/255., label

In [ ]:
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = test_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_data, test_data

In [ ]:
model_3 = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(300, 300, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model_3.summary()

In [ ]:
model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
history_3 = model_3.fit(train_data, epochs=5, verbose=1)

In [ ]:
pd.DataFrame(history_3.history).plot()

In [ ]:
model_3.evaluate(test_data)